In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
with open("d16_input.txt", "r") as f:
    lines = f.read().split("\n")
# lines = [(i[0], int(i[1:])) for i in lines]

In [3]:
lines[0], lines[-1]

('departure location: 27-374 or 395-974',
 '681,87,351,54,939,77,318,433,440,52,505,443,371,931,231,867,574,148,939,137')

In [4]:
def get_valid_list_for_filed(l):
    field = l.split(":")[0]
    t = l.split(":")[1].split("or")
    t1 = t[0].split("-")
    t2 = t[1].split("-")
    valid_list = list(range(int(t1[0].strip()), int(t1[1].strip()) + 1)) + list(
        range(int(t2[0].strip()), int(t2[1].strip()) + 1)
    )
    return field, valid_list

In [5]:
# Get field_dict, my ticket list and other ticket lists
field_dict = {}
for idx, l in enumerate(lines):
    if l != "":
        field, valid_list = get_valid_list_for_filed(l)
        field_dict[field] = valid_list
    else:
        break
my_ticket_li = [int(i) for i in lines[idx + 2].split(",")]
my_ticket_li
other_tickets = []
for ii in range(idx + 5, len(lines)):
    other_tickets.append([int(i) for i in lines[ii].split(",")])

In [6]:
# get all valid numbers
all_valid_num = []
for i in field_dict.values():
    all_valid_num.extend(i)
all_valid_num = set(all_valid_num)

In [7]:
# count unvalid numbers
all_num = np.array(other_tickets).flatten()
sum([i for i in all_num if i not in all_valid_num])

26941

In [8]:
# Get all valid tickets
valid_tickets = [i for i in other_tickets if pd.Series(i).isin(all_valid_num).all()]

In [9]:
# include my ticket
all_valid_t = np.vstack((my_ticket_li, np.array(valid_tickets)))
all_valid_t

array([[103,  79,  61, ...,  73, 107, 137],
       [473, 926, 599, ..., 450, 177, 113],
       [110,  74, 420, ..., 462, 837, 822],
       ...,
       [684, 520, 357, ..., 837, 255,  96],
       [287, 538, 856, ..., 117,  59, 906],
       [681,  87, 351, ..., 148, 939, 137]])

In [10]:
# Get initial possible fields
res = []
for i in range(all_valid_t.shape[1]):
    new_row = []
    for k, v in field_dict.items():
        possible = pd.Series(all_valid_t[:, i]).isin(field_dict[k]).all()
        if possible:
            new_row.append(k)
    res.append(new_row)

In [11]:
# find the only solution, unitl no more to find
ans_dict = {}
while True:
    new_res = res.copy()
    flag = True
    for idx, i in enumerate(res):
        if len(i) == 1:
            flag = False
            name = i[0]
            ans_dict[name] = idx
            for j in new_res:
                if name in j:
                    j.remove(name)
    res = new_res.copy()
    if flag:
        break
ans_dict

{'arrival station': 6,
 'zone': 0,
 'row': 12,
 'route': 10,
 'price': 8,
 'train': 3,
 'departure time': 1,
 'departure location': 13,
 'departure track': 5,
 'departure platform': 2,
 'departure date': 15,
 'departure station': 19,
 'wagon': 9,
 'seat': 7,
 'class': 14,
 'arrival track': 4,
 'duration': 18,
 'arrival platform': 17,
 'arrival location': 16,
 'type': 11}

In [12]:
# calculate the output
output = 1
for k, v in ans_dict.items():
    if k[: len("departure")] == "departure":
        output *= my_ticket_li[v]
output

634796407951